# Problem Set #7
### MACS 30100, Dr Evans
### Xi Chen

In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn import svm
import warnings
warnings.filterwarnings('ignore')

In [3]:
auto = pd.read_csv('Auto.csv', na_values='?')
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [4]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
mpg             397 non-null float64
cylinders       397 non-null int64
displacement    397 non-null float64
horsepower      392 non-null float64
weight          397 non-null int64
acceleration    397 non-null float64
year            397 non-null int64
origin          397 non-null int64
name            397 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 28.0+ KB


In [5]:
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      5
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [6]:
# Drop NAs
auto = auto.dropna()
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [7]:
# Create a binary variable mpg_high
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0).astype('category')
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0


In [8]:
X = auto[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = auto['mpg_high'].values

## (a) Logistic Regression

In [9]:
k = 4
kf = KFold(n_splits = k, shuffle = True, random_state = 15)
kf.get_n_splits(X)

LogRegress_MSE = np.zeros(k)
LogRegress_error_0 = np.zeros(k)
LogRegress_error_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum() 
    LogRegress_MSE[k_ind] = error_all_class
    LogRegress_error_0[k_ind] = error_0
    LogRegress_error_1[k_ind] = error_1
    print('When k index =', k_ind)
    print('The MSE for the test set', k_ind, 'is', error_all_class) 
    print('\n', classification_report(y_test, y_pred, digits=4))
    print('The error rate for category 0 is', error_0)
    print('The error rate for category 1 is', error_1)
    print('\n')
    k_ind += 1

When k index = 0
The MSE for the test set 0 is 0.09183673469387756

              precision    recall  f1-score   support

          0     0.9423    0.8909    0.9159        55
          1     0.8696    0.9302    0.8989        43

avg / total     0.9104    0.9082    0.9084        98

The error rate for category 0 is 0.057692307692307696
The error rate for category 1 is 0.13043478260869565


When k index = 1
The MSE for the test set 1 is 0.10204081632653061

              precision    recall  f1-score   support

          0     0.8776    0.9149    0.8958        47
          1     0.9184    0.8824    0.9000        51

avg / total     0.8988    0.8980    0.8980        98

The error rate for category 0 is 0.12244897959183673
The error rate for category 1 is 0.08163265306122448


When k index = 2
The MSE for the test set 2 is 0.1326530612244898

              precision    recall  f1-score   support

          0     0.8478    0.8667    0.8571        45
          1     0.8846    0.8679    0.87

In [10]:
print('Report the average MSE and the average error rates:\n')
print('The average MSE across 4 test sets is', LogRegress_MSE.mean())
print('The average error rate for category 0 is', LogRegress_error_0.mean())
print('The average error rate for category 1 is', LogRegress_error_1.mean())

Report the average MSE and the average error rates:

The average MSE across 4 test sets is 0.10714285714285715
The average error rate for category 0 is 0.09470670705864986
The average error rate for category 1 is 0.11822664912727025


## (b) Random Forest 

In [11]:
RandomForest = RandomForestClassifier(n_estimators = 20, max_features = 2, bootstrap = True, 
                                      oob_score = True, random_state = 25)
RandomForest.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [12]:
oob_prediction = RandomForest.oob_decision_function_.T[1]
RandomForest_MSE = pd.DataFrame({'pred' : oob_prediction, 'yvals': y})
RandomForest_MSE['pred'] = RandomForest_MSE['pred'].apply(lambda x: 1 if x >= 0.5 else 0)
RF_error = (RandomForest_MSE['pred'] != RandomForest_MSE['yvals']).apply(int)
print('The MSE of the model is', RF_error.mean())
print('\n',classification_report(RandomForest_MSE['yvals'], RandomForest_MSE['pred'], digits=4))
RandomForest_error_0 = ((RandomForest_MSE['pred'] == 0) * RF_error).sum() / (RandomForest_MSE['pred'] == 0).sum() 
RandomForest_error_1 = ((RandomForest_MSE['pred'] == 1) * RF_error).sum() / (RandomForest_MSE['pred'] == 1).sum() 
print('The error rate for category 0 is', RandomForest_error_0.mean())
print('The error rate for category 1 is', RandomForest_error_1.mean())

The MSE of the model is 0.07142857142857142

              precision    recall  f1-score   support

          0     0.9421    0.9133    0.9275       196
          1     0.9158    0.9439    0.9296       196

avg / total     0.9290    0.9286    0.9286       392

The error rate for category 0 is 0.05789473684210526
The error rate for category 1 is 0.08415841584158416


## (c) Support Vector Machines

In [13]:
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(X)
svm_MSE = np.zeros(k)
svm_error_0 = np.zeros(k)
svm_error_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('When k index =', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc = svm.SVC(kernel='rbf', C=1, gamma = 0.2)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    svm_MSE[k_ind] = error_all_class
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    svm_error_0[k_ind] = error_0
    svm_error_1[k_ind] = error_1
    print('The MSE for the test set', k_ind, 'is', error_all_class) 
    print('\n', classification_report(y_test, y_pred, digits=4))  
    print('The error rate for category 0 is', error_0)
    print('The error rate for category 1 is', error_1)
    print('\n')
    k_ind += 1

When k index = 0
The MSE for the test set 0 is 0.5408163265306123

              precision    recall  f1-score   support

          0     1.0000    0.0364    0.0702        55
          1     0.4479    1.0000    0.6187        43

avg / total     0.7578    0.4592    0.3109        98

The error rate for category 0 is 0.0
The error rate for category 1 is 0.5520833333333334


When k index = 1
The MSE for the test set 1 is 0.5204081632653061

              precision    recall  f1-score   support

          0     0.4796    1.0000    0.6483        47
          1     0.0000    0.0000    0.0000        51

avg / total     0.2300    0.4796    0.3109        98

The error rate for category 0 is 0.5204081632653061
The error rate for category 1 is nan


When k index = 2
The MSE for the test set 2 is 0.5204081632653061

              precision    recall  f1-score   support

          0     0.4688    1.0000    0.6383        45
          1     1.0000    0.0377    0.0727        53

avg / total     0.7561 

In [14]:
print('Report the average MSE and the average error rates:\n')
print('The average MSE across 4 test sets is', svm_MSE.mean())
print('The average error rate for category 0 is', svm_error_0.mean())
svm_error_1 = svm_error_1[~np.isnan(svm_error_1)]
print('The average error rate for category 1 is', svm_error_1.mean())

Report the average MSE and the average error rates:

The average MSE across 4 test sets is 0.5076530612244898
The average error rate for category 0 is 0.38119411070879966
The average error rate for category 1 is 0.1840277777777778


## (d) Compare Logistic Regression, Random Forest, and SVM

In [15]:
print('Logistic Regression:\n')
print('The average MSE across 4 test sets is', LogRegress_MSE.mean())
print('The average error rate for category 0 is', LogRegress_error_0.mean())
print('The average error rate for category 1 is', LogRegress_error_1.mean())
print('\n')

print('Random Forest:\n')
print('The MSE of the model is', RF_error.mean())
print('The error rate for category 0 is', RandomForest_error_0.mean())
print('The error rate for category 1 is', RandomForest_error_1.mean())
print('\n')

print('Support Vector Machines:\n')
print('The average MSE across 4 test sets is', svm_MSE.mean())
print('The average error rate for category 0 is', svm_error_0.mean())
print('The average error rate for category 1 is', svm_error_1.mean())
print('\n')

Logistic Regression:

The average MSE across 4 test sets is 0.10714285714285715
The average error rate for category 0 is 0.09470670705864986
The average error rate for category 1 is 0.11822664912727025


Random Forest:

The MSE of the model is 0.07142857142857142
The error rate for category 0 is 0.05789473684210526
The error rate for category 1 is 0.08415841584158416


Support Vector Machines:

The average MSE across 4 test sets is 0.5076530612244898
The average error rate for category 0 is 0.38119411070879966
The average error rate for category 1 is 0.1840277777777778




From the results above, we can see that the Random Forest model has the lowest MSE and error rates for both category 0 and 1; the Logistic Regression model has the second best performance; the SVM has the worst performance. Hence, the Random Forest model seems to be the best predictor of mpg_high. There are several potential reasons for these results. For example, both the Logistic Regression model and the SVM use k-fold cross validation to compute the average MSE, while the Random Forest model uses the oob prediction to compute the MSE. The different method to compute MSE may lead to different results.